In [ ]:
# train_model_colab.ipynb
# This is a Google Colab notebook - save with .ipynb extension

# Title: Offline Learning - Risk Model Training
# Description: Train ML model for student risk prediction
# Author: Your Name
# Date: 2024

# ============================================
# 1. SETUP & INSTALL DEPENDENCIES
# ============================================

!pip install scikit-learn==1.3.2 pandas==2.1.3 numpy==1.26.2 joblib==1.3.2

# ============================================
# 2. MOUNT GOOGLE DRIVE (Save models/data)
# ============================================

from google.colab import drive
drive.mount('/content/drive')

# Create project folder in Drive
import os
project_path = '/content/drive/MyDrive/offline_learning'
os.makedirs(project_path, exist_ok=True)

# ============================================
# 3. DOWNLOAD YOUR PROJECT FILES
# ============================================

# Option A: Upload manually
print("Please upload these files to Colab:")
print("1. Backend/ml_model.py")
print("2. train_model.py")
print("3. Any dataset files")

# Option B: Clone from GitHub (if you push first)
# !git clone https://github.com/YOUR_USERNAME/offline-learning.git
# %cd offline-learning

# ============================================
# 4. TRAIN THE RISK PREDICTION MODEL
# ============================================

# Import your ML model class
import sys
sys.path.append('/content')

# Create a simplified training script
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

print("📊 Training Risk Prediction Model...")

# ============================================
# 5. CREATE SAMPLE DATA (Replace with your actual data)
# ============================================

# Generate synthetic training data
np.random.seed(42)
n_samples = 1000

# Features based on your ml_model.py
data = {
    'attendance_rate': np.random.uniform(40, 100, n_samples),
    'avg_score': np.random.uniform(30, 95, n_samples),
    'study_consistency': np.random.uniform(1, 5, n_samples),
    'activity_completion_rate': np.random.uniform(50, 100, n_samples)
}

df = pd.DataFrame(data)

# Create risk labels based on rules (similar to your logic)
def calculate_risk(row):
    if row['attendance_rate'] < 50 or row['avg_score'] < 40:
        return 'high'
    elif row['attendance_rate'] < 70 or row['avg_score'] < 60:
        return 'medium'
    else:
        return 'low'

df['risk_level'] = df.apply(calculate_risk, axis=1)

print(f"Dataset created: {len(df)} samples")
print("Risk level distribution:")
print(df['risk_level'].value_counts())

# ============================================
# 6. TRAIN MODEL
# ============================================

# Features and target
features = ['attendance_rate', 'avg_score', 'study_consistency', 'activity_completion_rate']
X = df[features]
y = df['risk_level']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    class_weight='balanced'
)

model.fit(X_train_scaled, y_train)

# Evaluate
train_score = model.score(X_train_scaled, y_train)
test_score = model.score(X_test_scaled, y_train)

print(f"\n✅ Model trained successfully!")
print(f"   Training accuracy: {train_score:.2%}")
print(f"   Testing accuracy: {test_score:.2%}")
print(f"   Classes: {model.classes_}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n📈 Feature Importance:")
print(feature_importance)

# ============================================
# 7. SAVE MODEL TO GOOGLE DRIVE
# ============================================

# Save the model (same format as your risk_model.pkl)
model_data = {
    'model': model,
    'scaler': scaler,
    'features': features
}

model_path = f'{project_path}/risk_model_colab.pkl'
joblib.dump(model_data, model_path)

print(f"\n💾 Model saved to Google Drive:")
print(f"   {model_path}")

# ============================================
# 8. DOWNLOAD MODEL TO LOCAL COMPUTER
# ============================================

# Download the model file
from google.colab import files
files.download(model_path)

print("\n⬇️ Model downloaded to your computer!")
print("Rename it to 'risk_model.pkl' and replace your local file.")

# ============================================
# 9. TEST PREDICTIONS
# ============================================

print("\n🧪 Testing predictions...")

# Test with sample student
test_student = {
    'attendance_rate': 85,
    'avg_score': 75,
    'study_consistency': 3.5,
    'activity_completion_rate': 90
}

# Prepare input
X_test = pd.DataFrame([test_student])[features]
X_test_scaled = scaler.transform(X_test)

# Predict
prediction = model.predict(X_test_scaled)[0]
probabilities = model.predict_proba(X_test_scaled)[0]

print(f"\nTest Student:")
for key, value in test_student.items():
    print(f"  {key}: {value}")

print(f"\nPredicted Risk: {prediction}")
for cls, prob in zip(model.classes_, probabilities):
    print(f"  P({cls}) = {prob:.2%}")

# ============================================
# 10. VISUALIZATIONS
# ============================================

# Optional: Add charts if you want
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Feature importance chart
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance, x='importance', y='feature')
    plt.title('Feature Importance for Risk Prediction')
    plt.tight_layout()
    plt.savefig(f'{project_path}/feature_importance.png')
    plt.show()
    
    print(f"\n📊 Visualization saved to Google Drive")
    
except:
    print("\n📊 Install matplotlib and seaborn for visualizations:")
    print("!pip install matplotlib seaborn")

# ============================================
# 11. NEXT STEPS
# ============================================

print("\n" + "="*50)
print("🎉 TRAINING COMPLETE!")
print("="*50)
print("\nNext steps:")
print("1. Download the model file (risk_model_colab.pkl)")
print("2. Rename it to 'risk_model.pkl'")
print("3. Replace your local risk_model.pkl file")
print("4. Restart your FastAPI server")
print("5. Test the new model at: http://localhost:8000/risk")
print("\nTo share this Colab:")
print("File → Share → Get shareable link")